In [1]:
%cd drive/MyDrive/FastSpeech2-1/

/content/drive/MyDrive/FastSpeech2-1


In [5]:
!pip install textgrid

In [17]:
import numpy as np
import librosa
import textgrid
import glob
from utils.hparams import HParam
from utils.util import get_files
hp = HParam("configs/default.yaml")

In [39]:
from sklearn.model_selection import train_test_split

def split_dataset(dataset):
    """Разделяем выборку"""
    data_train, data_validate = \
        train_test_split(dataset, test_size=0.05, shuffle=False)
    return len(data_train), len(data_validate)

In [9]:
files = get_files('/content/drive/MyDrive/FastSpeech2/preprocessed_data/SOVA/TextGrid/', extension=".TextGrid")

In [31]:
((files[0].split(".")[0]) + ".wav").split('/')[-1]

'003273.wav'

In [40]:
selected_sep = split_dataset(files)
selected_sep

(3700, 195)

In [41]:
for i in range(0, len(files)):
    tg = textgrid.TextGrid.fromFile(files[i])
    time_list = []
    phonemes = []
    words = []
    diff_list = []
    frames = []

    file_name = ((files[i].split(".")[0]) + ".wav").split('/')[-1]
    time_list.append(0.0)
    for interval in tg[1]:
        time_list.append(interval.maxTime)
        if (interval.mark == "sp"):
            interval.mark = "pau"
        phonemes.append(interval.mark)
    frames = librosa.core.time_to_frames(
        np.array(time_list), sr = hp.audio.sample_rate, 
        hop_length = hp.audio.hop_length,  n_fft= hp.audio.n_fft)
    
    for j in range(1, frames.shape[0]):
        diff_list.append(frames[j] - frames[j-1])
    for word in (tg[0]):
        words.append(word.mark)
    text = ' '.join([str(elem) for elem in words])
    content = text.strip() + "|" + ' '.join([str(elem) for elem in frames]) + "|" + ' '.join([str(elem) for elem in diff_list]) + "|" + ' '.join([str(elem) for elem in phonemes]) + "|" + file_name + "\n"
    if (i < selected_sep[1]):
        with open("filelists/" + "valid.txt", "a") as f:
            f.write(content)
    else:
        with open("filelists/" + "train.txt", "a") as f1:
            f1.write(content)